In [1]:
import os

In [2]:
%pwd

'c:\\Users\\tushir sahu\\OneDrive\\Desktop\\MLOPS\\MLOPS_\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
  root_dir: Path
  source_url: str
  local_data_file: Path
  unzip_dir: Path

In [5]:
from DeepLearning.constants import *
from DeepLearning.utils.common import read_yaml,create_directories 

In [6]:
class  ConfigurationManager:
    def __init__(self,
        config_path = CONFIG_FILE_PATH,
        params_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:

        config = self.config.data_ingestion
        create_directories([config.root_dir])
        return DataIngestionConfig(
            root_dir = (config.root_dir),
            source_url = (config.source_url),
            local_data_file = (config.local_data_file),
            unzip_dir = (config.unzip_dir)
        )

In [7]:
import urllib.request as request
import zipfile
from DeepLearning import logger
from DeepLearning.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(url=self.config.source_url,filename=self.config.local_data_file)
            logger.info(f"{filename} downloaded successfully with info {headers}")
        else : logger.info(f"File already exits with size: {get_size(Path(self.config.local_data_file))}")
            # os.makedirs(self.config.root_dir)
    def extract_zip_file(self):
        unzip_path= (self.config.unzip_dir)
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)

In [11]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-20 00:59:57,952: INFO: common: yaml file loaded from config\config.yaml]
[2023-10-20 00:59:57,954: INFO: common: yaml file loaded from params.yaml]
[2023-10-20 00:59:57,955: INFO: common: created directory at: artifacts]
[2023-10-20 00:59:57,956: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-20 01:00:00,428: INFO: 3955871954: artifacts/data_ingestion/data.zip downloaded successfully with info Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3128:3D49F0:6E8BD:AD296:653183B5
Accept-Ranges: bytes
Date: Thu, 19 Oct 2023 19:29:59 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100031-HYD
X-Cache: MISS
X-Cache-Hit